In [4]:
import sys

sys.path.append("..")
# sys.path.append("..")
from matplotlib import cm, use
import math
import time
from pupil_analysis_func import Main
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import analysis_utils as utils
from copy import deepcopy as copy
from behaviour_analysis import TDAnalysis
import math
import pickle
from datetime import datetime, timedelta
from scipy.signal import find_peaks, find_peaks_cwt
from scipy.stats import sem
from pupil_analysis_func import batch_analysis, plot_traces, get_subset, glm_from_baseline, PupilEventConditions
from plotting_functions import plot_sound_vbars, plot_ts_var, FigureObj
from pathlib import Path

In [5]:
plt.ioff()
paradigm = ['familiarity']
pkldir = r'D:\bonsai\offline_data'
pkl2use = os.path.join(pkldir,'mouse_hf_2309_batch_w_canny_DEC_dates_fam_2d_90Hz_hpass01_lpass4hanning025_TOM.pkl')
run = Main(pkl2use, (-1.0, 3.0), figdir=rf'figures',fig_ow=False)
pmetric2use = ['diameter_2d_zscored','dlc_radii_a_zscored','dlc_EW_zscored','dlc_EW_normed']

do_baseline = True  # 'rawsize' not in pkl2use
run.add_pretone_dt()
run.aligned = {}
align_pnts = ['ToneTime','Reward','Gap_Time']

animals2plot = run.labels
dates2plot = run.dates
stages = [3]
run.add_stage3_05_order()
run.add_rolling_mean('Tone_Position',10)
dateconds = run.dates
run.add_diff_col_dt('Trial_Outcome')

In [6]:
condition_keys = ['p_rate','p_rate_local']
condition_keys_canny = [f'{e}_canny' for e in condition_keys]
# aligned_pklfile = r'pickles\fm_fam_aligned_nohpass.pkl'
# aligned_pklfile = r'pickles\DO54_62_aligned_notrend.pkl'
# aligned_pklfile = r'mouse_hf_2305_batch_no_canny_fam_hpass015.pkl'
# aligned_pklfile = r'mouse_hf_2309_batch_w_canny_fam_hpass01.pkl'
aligned_pklfile = r'mouse_hf_2309_batch_DEC_dates_w_canny_fam_hpass01_align.pkl'
# aligned_pklfile = r'C:\bonsai\gd_analysis\pickles\normdev_2305cohort_aligned.pkl'
aligned_ow = False
conditions_class = PupilEventConditions()
list_cond_filts = conditions_class.all_filts
for sess in run.data:
    run.data[sess].trialData['Offset'] = run.data[sess].trialData['Offset'].astype(float) + 0.0
if os.path.isfile(aligned_pklfile) and aligned_ow is False:
    with open(aligned_pklfile,'rb') as pklfile:
        run.aligned = pickle.load(pklfile)
        keys = [[e] for e in run.aligned.keys()]
else:
    conditions_class.get_condition_dict(run, condition_keys,stages,)  # 'a1'
    conditions_class.get_condition_dict(run, condition_keys, stages,
                                        pmetric2use='canny_raddi_a_zscored', key_suffix='_canny')

processing condition key:   0%|          | 0/2 [00:00<?, ?it/s]

none filts = ['c1', 'stage3']
[0, 0, 0, 0]



100%|██████████| 78/78 [00:33<00:00,  2.34it/s]

processing condition key:  50%|█████     | 1/2 [00:34<00:34, 34.80s/it]

none filts = ['c1', 'stage3']
[0, 0, 0, 0, 0, 0]



100%|██████████| 78/78 [00:33<00:00,  2.32it/s]

processing condition key:   0%|          | 0/2 [00:00<?, ?it/s]

none filts = ['c1', 'stage3']
[0, 0, 0, 0]



100%|██████████| 78/78 [00:33<00:00,  2.36it/s]

processing condition key:  50%|█████     | 1/2 [00:34<00:34, 34.57s/it]

none filts = ['c1', 'stage3']
[0, 0, 0, 0, 0, 0]



100%|██████████| 78/78 [00:33<00:00,  2.31it/s]

processing condition key: 100%|██████████| 2/2 [01:09<00:00, 34.91s/it]


In [7]:
fig1 = FigureObj()
plt.close('all')

In [8]:
fig1 = FigureObj()

In [9]:
p_rate_dates=run.dates
# prate_muscimol_dates = ['230918','230920','230927','230929','231002','231030','231103']
prate_muscimol_dates = ['230918','230920','230927','230929','231002','231030','231103',
                        '231128','231201','231206']
# prate_saline_dates = ['230928','231024','231027','231102']
prate_saline_dates = ['230928','231024','231027','231102','231204']
prate_control_dates = [d for d in p_rate_dates if d not in prate_muscimol_dates+prate_saline_dates]
prate_control_dates = [d for d in prate_control_dates if all(int(d) - np.array(prate_muscimol_dates).astype(int) != -1)]
# prate_control_dates.remove('231031')
muscimol_analysis_dfs = []
for subset_ix, (subset_dates,subset_name) in enumerate(zip([prate_muscimol_dates,prate_control_dates,prate_saline_dates],
                                                           ['muscimol', 'control','saline'])):
    run.subsets[f'{subset_name}_2patt'] = get_subset(run, run.aligned, 'p_rate_local_canny',{'date':subset_dates,'name':['DO71','DO72','DO75']},
                                                     events=list_cond_filts['p_rate_local'][1],
                                                     beh=f'{align_pnts[0]} onset',
                                                     plttitle=f'Response to pattern onset {subset_name}',
                                                     plttype='ts',
                                                     ylabel='zscored pupil size',
                                                     xlabel=f'Time since pattern onset (s)',
                                                     exclude_idx=(1, 2, 3),
                                                     alt_cond_names=['rare','frequent','none']
                                                     )
    muscimol_analysis_dfs.append(run.subsets[f'{subset_name}_2patt'][2])

length input lists 3
length input lists 3
length input lists 3


In [10]:
print(prate_control_dates)

['230913', '230914', '230915', '231023', '231025', '231031']


In [29]:
drug_df_dicts = []
for subset_name in ['muscimol', 'control','saline']:
    dfss = []
    for df, cond in zip(run.subsets[f'{subset_name}_2patt'][2],['rare','frequent']):
        # print(df.description)
        df = df.copy()
        df.columns = np.round(np.linspace(-1,2.99,400),2)
        sessname_idx = df.index.to_frame()
        sessname_idx = sessname_idx.apply(lambda row: f'{row[1]}_{row[2]}',axis=1)
        trial_times = sessname_idx.index.get_level_values('time').to_series().apply(lambda x: x.timestamp())/1e3
        df.index = pd.MultiIndex.from_arrays([trial_times,sessname_idx.reset_index(drop=False).index.values,sessname_idx],
                                             names=['time','trial','sess'])
        dfss.append(df)
    df_dict = {cond: cond_df for cond, cond_df in zip(['rare','frequent'],dfss)}
    drug_df_dicts.append(copy(df_dict))
by_cond_df = {cond:pd.concat([subset_dict[cond] for subset_dict in drug_df_dicts]) for cond in ['rare','frequent']}
pkl_path = r'X:\Dammy\pupil_data\musc_sept23_A_by_cond.pkl'
with open(pkl_path,'wb') as pklfile:
    pickle.dump(by_cond_df,pklfile)

In [12]:
np.round(np.linspace(-1,2.99,400
        ),2
        )

array([-1.  , -0.99, -0.98, -0.97, -0.96, -0.95, -0.94, -0.93, -0.92,
       -0.91, -0.9 , -0.89, -0.88, -0.87, -0.86, -0.85, -0.84, -0.83,
       -0.82, -0.81, -0.8 , -0.79, -0.78, -0.77, -0.76, -0.75, -0.74,
       -0.73, -0.72, -0.71, -0.7 , -0.69, -0.68, -0.67, -0.66, -0.65,
       -0.64, -0.63, -0.62, -0.61, -0.6 , -0.59, -0.58, -0.57, -0.56,
       -0.55, -0.54, -0.53, -0.52, -0.51, -0.5 , -0.49, -0.48, -0.47,
       -0.46, -0.45, -0.44, -0.43, -0.42, -0.41, -0.4 , -0.39, -0.38,
       -0.37, -0.36, -0.35, -0.34, -0.33, -0.32, -0.31, -0.3 , -0.29,
       -0.28, -0.27, -0.26, -0.25, -0.24, -0.23, -0.22, -0.21, -0.2 ,
       -0.19, -0.18, -0.17, -0.16, -0.15, -0.14, -0.13, -0.12, -0.11,
       -0.1 , -0.09, -0.08, -0.07, -0.06, -0.05, -0.04, -0.03, -0.02,
       -0.01,  0.  ,  0.01,  0.02,  0.03,  0.04,  0.05,  0.06,  0.07,
        0.08,  0.09,  0.1 ,  0.11,  0.12,  0.13,  0.14,  0.15,  0.16,
        0.17,  0.18,  0.19,  0.2 ,  0.21,  0.22,  0.23,  0.24,  0.25,
        0.26,  0.27,

In [30]:
musc_sal_ctrl_tsplot = plt.subplots(figsize=(9,7))
rare_freq_delta_tsplot = plt.subplots(figsize=(9,7))
delta_dfsss = []
for cond_i, (cond_dfs,cond_name,ls) in enumerate(zip(muscimol_analysis_dfs,['muscimol','control',],['-','--',':'])):
    rare_df, freq_df, none_df = copy(cond_dfs)
    none_df.index = none_df.index.droplevel('time')
    for df_i, (df,df_name) in enumerate(zip([rare_df,freq_df],['rare','frequent'])):
        df.index = df.index.droplevel('time')
        # for u_idx in df.index.unique():
            # df.loc[u_idx] = df.loc[u_idx] - none_df.loc[u_idx].median(axis=0)
        musc_sal_ctrl_tsplot[1].plot(none_df.columns, df.mean(axis=0),
                                     c=f'C{df_i}',ls=ls,label=f'{cond_name}: {df_name}')
    delta_dfs = np.array([rare_df.loc[u_idx].mean(axis=0)-freq_df.loc[u_idx].mean(axis=0) for u_idx in rare_df.index.unique() if u_idx in freq_df.index])
    delta_dfsss.append(delta_dfs)
    # delta_means = [(rare_df.loc[u_idx]-freq_df.loc[u_idx]).mean(axis=0) for u_idx in rare_df.index.unique() if u_idx in freq_df.index]
    rare_freq_delta_tsplot[1].plot(none_df.columns, delta_dfs.mean(axis=0),
                                   label=f'{cond_name}',c='black',ls=ls)
    # plot_ts_var(none_df.columns,np.array(delta_dfs),f'black',rare_freq_delta_tsplot[1])
    delta_sem = sem(delta_dfs,axis=0,nan_policy='omit')
    rare_freq_delta_tsplot[1].fill_between(none_df.columns,delta_dfs.mean(axis=0)-delta_sem,delta_dfs.mean(axis=0)+delta_sem,
                                           fc='k', alpha=0.1)

In [33]:
savedir = Path(r'X:\Dammy\pupil_data')
[np.save(savedir/tag,arr) for arr,tag in zip(delta_dfsss,['sept_musc.npy','sept_control.npy'])]

[None, None]

In [28]:
print(muscimol_analysis_dfs[0][0].index.get_level_values('date').unique())
print(muscimol_analysis_dfs[1][0].index.get_level_values('date').unique())
print(muscimol_analysis_dfs[2][0].index.get_level_values('date').unique())

Index(['230918', '230920', '230927', '230929', '231002', '231128', '231206',
       '231030', '231103', '231201'],
      dtype='object', name='date')
Index(['230913', '230914', '230915', '231025', '231023', '231031'], dtype='object', name='date')
Index(['230928', '231024', '231027', '231204', '231102'], dtype='object', name='date')


In [14]:
figdir = Path(r'C:\Users\Danny\OneDrive - University College London\for_cosyne')

In [15]:
rare_freq_delta_tsplot[1].legend()
[rare_freq_delta_tsplot[1].axvspan(t,t+0.15,fc='k',alpha=0.1) for t in np.arange(0,1,.25)]
rare_freq_delta_tsplot[1].set_ylabel('rare response - frequent response')
rare_freq_delta_tsplot[1].set_xlabel('Time since pattern onset (s)')
rare_freq_delta_tsplot[1].axvline(0,c='k',ls='--')
rare_freq_delta_tsplot[0].set_constrained_layout('constrained')
# rare_freq_delta_tsplot[0].show()
rare_freq_delta_tsplot[0].set_size_inches(9,7)

rare_freq_delta_tsplot[0].savefig(figdir/'rare_freq_delta_ts.svg')


In [16]:
run.subsets['muscimol_2patt'][0].set_size_inches(9,7)

In [17]:
run.subsets['muscimol_2patt'][1].set_title('')
run.subsets['muscimol_2patt'][0].set_constrained_layout('constrained')
run.subsets['muscimol_2patt'][0].savefig(figdir/'muscimol_2patt.svg')